# Client Segmentation Clustering

We have a dataset about the stocks management of a company. We have data about orders made by different clients and also other type of transactions like shipipment orders, bank transactions stock balancing transactions, etc.

We need to focus only on the proper customer transactions and group the customers in certain clusters based on their behaviours

### Importing the dataset

We need to change the default encondig because of some charactrs in the dataset

In [46]:
import pandas as pd

df = pd.read_csv('data.csv', encoding='unicode_escape')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


We want to see how many rows and columns we have

- we have 541909 rows and 8 columns

In [47]:
df.shape

(541909, 8)

We want to see what types of data we have

1. We notice that we have some columns that are normally numeric ones like InvoiceNo, StockCode that are now as objects because they also contain letters, we will need to somehow convert these to numeric type

2. We also notice that InvoiceDate is object and we will convert it to datetype

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


### Data cleaning

Handling null values:

We want to see the features with nulls

Since we want to observe the behaviour of customers we need to get rid of the data without 'Customer ID'

In [49]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

By deleting all the 'CustomerID' rows with null values it will remove also all the rows from 'Description' that have null values because the rows match

In [50]:
df.dropna(subset=['CustomerID'], inplace=True, axis=0)
df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

Checking duplicates

We see that there are duplicates but it seems that the system allows buying the same product multiple times on an order

We see that on the same InvoiceNO '536381' we have the same product '71270' added with different quantities - this makes us think that it is possible to have the same product mutiple times on the same order with the same or different 'Quantity'

In [65]:
duplicated_rows = df[df.duplicated(keep=False)]
duplicated_rows = duplicated_rows.sort_values(by=['InvoiceNo', 'StockCode'])
duplicated_rows.head()

# Group by 'orderID' and 'product_code', then filter based on multiple occurrences with different quantities
duplicate_orders = df[df.duplicated(subset=['InvoiceNo', 'StockCode'], keep=False)]

print("Orders with the same product_code brought multiple times with different quantities:")
duplicate_orders.head()

Orders with the same product_code brought multiple times with different quantities:


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
113,536381,71270,PHOTO CLIP LINE,1,12/1/2010 9:41,1.25,15311.0,United Kingdom
125,536381,71270,PHOTO CLIP LINE,3,12/1/2010 9:41,1.25,15311.0,United Kingdom
483,536409,90199C,5 STRAND GLASS NECKLACE CRYSTAL,3,12/1/2010 11:45,6.35,17908.0,United Kingdom
485,536409,22111,SCOTTIE DOG HOT WATER BOTTLE,1,12/1/2010 11:45,4.95,17908.0,United Kingdom
489,536409,22866,HAND WARMER SCOTTY DOG DESIGN,1,12/1/2010 11:45,2.10,17908.0,United Kingdom
